In [3]:
import json
import tweepy
import pandas as pd
from tweepy import OAuthHandler
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time
import re

In [4]:
analyzer = SentimentIntensityAnalyzer()

In [42]:
analyzer = SentimentIntensityAnalyzer()
res = analyzer.polarity_scores('Vader is a bad guy by I love this analyzer.')
res

{'neg': 0.255, 'neu': 0.438, 'pos': 0.307, 'compound': 0.1779}

In [5]:
con_key = 'uz4wSx2vt2PlKbWCOlIpKbBZo'
con_secret = 'lq61FxYRvJPwLjjRlUf0rSTsdXf86iijHYeGfrUdi7EhZzWsWv'
acc_token = '1038510225786122247-zLnTCjChQ0rLUHevueL0CN4ygdY1Me'
acc_token_secret = 'W7lqUevzXRpzI4zQA6niMDR85r9SCo3sCQGaPA4wg6DS6'

In [ ]:
analyzer = SentimentIntensityAnalyzer()
res = analyzer.polarity_scores('Vader is a bad guy by I love this analyzer.')

In [6]:
auth = OAuthHandler(con_key, con_secret)
auth.set_access_token(acc_token, acc_token_secret)
new = tweepy.API(auth)

In [40]:
results = new.search(q="@FLGovScott" , count = 2 , lang='en' ,  geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')



In [38]:
results

[Status(_api=<tweepy.api.API object at 0x111aedeb8>, _json={'created_at': 'Mon Oct 01 00:06:50 +0000 2018', 'id': 1046551977478565890, 'id_str': '1046551977478565890', 'full_text': 'Can you say \'endorsement\'? @SenBillNelson\'s #flsen camp announces he\'ll join PR Gov. @ricardorossello &amp; former Govs. Padilla &amp; Rossello for a "special campaign announcement" Monday at Nelson HQ in Orlando. The appearance comes two weeks after @FLGovScott visited P.R.', 'truncated': False, 'display_text_range': [0, 275], 'entities': {'hashtags': [{'text': 'flsen', 'indices': [44, 50]}], 'symbols': [], 'user_mentions': [{'screen_name': 'SenBillNelson', 'name': 'Senator Bill Nelson', 'id': 20597460, 'id_str': '20597460', 'indices': [27, 41]}, {'screen_name': 'ricardorossello', 'name': 'Ricardo Rossello', 'id': 80013913, 'id_str': '80013913', 'indices': [85, 101]}, {'screen_name': 'FLGovScott', 'name': 'Rick Scott', 'id': 131546062, 'id_str': '131546062', 'indices': [251, 262]}], 'urls': []}, 'metad

In [41]:
for result in results:
    print(result.created_at)
    print('\n')
    print('1::: fullText:: ' , result.full_text.lower())
    if result.full_text[:2] == 'RT':
        print('Helllllloo')
    print('2::: geo:: ' , result.full_text[:2])
    print('3::: place:: ' , result.place)
    print('4::: coordinates:: ' , result.coordinates)
    print('5::: retweet Count::  ' , result.retweet_count)
    print('6::: retweeted:: ' , result.retweeted)
    print('7::: lang:: ' , result.lang)
    print('8::: ID:: ' , result.id_str)
    print(result.contributors)
    print('9::: TextRange:: ' , result.entities)
    print('10::: User Location:: ', result.user.location)
    print('9::: TextRange:: ' ,result.metadata)
    print('1000::: USER:: ' , result.user)
   # result.full_text

2018-10-01 00:10:07


1::: fullText::  rt @troykinsey: can you say 'endorsement'? @senbillnelson's #flsen camp announces he'll join pr gov. @ricardorossello &amp; former govs. padill…
Hellllll
2::: geo::  RT
3::: place::  None
4::: coordinates::  None
5::: retweet Count::   1
6::: retweeted::  False
7::: lang::  en
8::: ID::  1046552807208308736
None
9::: TextRange::  {'hashtags': [{'text': 'flsen', 'indices': [60, 66]}], 'symbols': [], 'user_mentions': [{'screen_name': 'TroyKinsey', 'name': 'Troy Kinsey', 'id': 43945628, 'id_str': '43945628', 'indices': [3, 14]}, {'screen_name': 'SenBillNelson', 'name': 'Senator Bill Nelson', 'id': 20597460, 'id_str': '20597460', 'indices': [43, 57]}, {'screen_name': 'ricardorossello', 'name': 'Ricardo Rossello', 'id': 80013913, 'id_str': '80013913', 'indices': [101, 117]}], 'urls': []}
10::: User Location::  St. Petersburg, FL
9::: TextRange::  {'iso_language_code': 'en', 'result_type': 'recent'}
1000::: USER::  User(_api=<tweepy.api.API object at 0x

## This is the Begining

In [166]:
bagOwords = ['election' , 'senate' , 'race' , 'november' , '6' , 'gop' , 'dnc' , 'win' , 'lose']
analyzer = SentimentIntensityAnalyzer()
def Senter(tweets , Cadidate , DF):
    for tweet in tweets:
        sentiment = '0'
        retweet = '0'
        relivent = 0
        
        if tweet.full_text[:2] == 'RT':
            retweet = '1'
        
        lower_txt = tweet.full_text.lower()
        for word in bagOwords:
            if word in lower_txt:
                relivent = 1
        
        if tweet.lang == 'en':
            #send to english sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        else:
            #send to this sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        DF = DF.append({'Candidate': '%s' % (Cadidate) , 'Time' : tweet.created_at ,
               'Text': tweet.full_text , 'UserLocation': tweet.user.location , 
               'Coordinates': tweet.coordinates , 'Retweeted_count': tweet.retweeted_count , 
               'IsReTweet': retweet ,'Language' : tweet.lang , 'Sentiment': sentiment ,
               'Relivent' : relivent } , ignore_index = True)

    return DF

In [156]:
#Keep in mind we are only pulling english tweets
def Searcher(Handle , Cadidate , GeoCode , DF):
    tweets = new.search(q='%s' % (Handle), lang = 'en' , count = consume_count  , geocode = '%s' % (GeoCode) , tweet_mode = 'extended')     
    if len(tweets) == 0:
        time.sleep(60 * 15)
        tweets = new.search(q='%s' % (Handle), lang = 'en' , count = consume_count ,  geocode = '%s' % (GeoCode) , tweet_mode = 'extended')    
    DF = Senter(tweets , Cadidate , DF)
    return DF

In [ ]:
columns = ['Candidate' , 'Time' , 'Text' , 'UserLocation' , 'Coordinates' , 'Retweeted_count' , 'IsReTweet' , 'Language' , 'Sentiment' , 'Relivent' ]
AZ_path_name = 'iCloud Drive/Desktop/Politter/AZOutput'
FL_path_name = 'iCloud Drive/Desktop/Politter/FLOutput'
FL_Radius = '27.967042,-81.471955,320km'
AZ_Radius = '34.383026,-111.694738,400km'
consume_count = 15
analyzer = SentimentIntensityAnalyzer()
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if time.localtime().tm_hour == 24:
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')
        
    FL = Searcher("@FLGovScott" , 'GOP:RICK SCOTT' , FL_Radius , FL)
    FL = Searcher("@SenBillNelson" , 'DEM:BILL NELSON' , FL_Radius , FL)
    
    AZ = Searcher("@RepMcSally" , 'GOP:MARTHA MCSALLY' , AZ_Radius , AZ)
    AZ = Searcher("@RepSinema" , 'DEM:KYRSTEN SINEMA' , AZ_Radius , AZ)
    

## END

## Testing

In [ ]:
columns = ['Candidate' , 'Time' , 'Text' , 'userLocation' , 'coordinates' , 'retweeted' , 'language' , 'Sentiment' ]
AZ_path_name = 'iCloud Drive/Desktop/Politter/AZOutput'
FL_path_name = 'iCloud Drive/Desktop/Politter/FLOutput'
consume_count = 1
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if time.localtime().tm_hour == 24:
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')
      
    
    #Florida Race
    Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Rick) == 0:
        time.sleep(60 * 15)
        Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Rick , 'GOP:RICK SCOTT' , FL)
    
    Bill = new.search(q="@SenBillNelson" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Bill) == 0:
        time.sleep(60 * 15)
        Bill = new.search(q="@SenBillNelson" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Bill , 'DEM:BILL NELSON' , FL)
    
    
    #Arazona Race
    Martha = new.search(q="@RepMcSally" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    if len(Martha) == 0:
        time.sleep(60 * 15)
        Martha = new.search(q="@RepMcSally" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    Senter(Martha , 'GOP:MARTHA MCSALLY' , AZ)
    
    Kyrsten =  new.search(q="@RepSinema" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    if len(Kyrsten) == 0:
        time.sleep(60 * 15)
    Kyrsten = new.search(q="@RepSinema" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    Senter(Kyrsten , 'DEM:KYRSTEN SINEMA' , AZ)

In [120]:
columns = ['Candidate' , 'Time' , 'Text' , 'userLocation' , 'coordinates' , 'retweeted' , 'language' , 'Sentiment' ]
AZ_path_name = 'iCloud Drive/Desktop/Politter/AZOutput'
FL_path_name = 'iCloud Drive/Desktop/Politter/FLOutput'
consume_count = 1
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if time.localtime().tm_hour == 24:
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')
    Searcher("@FLGovScott" , 'GOP:RICK SCOTT' , '27.967042,-81.471955,320km' , FL)

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                                                Text userLocation coordinates  \
0  RT @FLGovScott: Hurricane season doesn’t end u...                     None   

  retweeted language                                          Sentiment  \
0     False      NaN  {'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'comp...   

  Language  
0       en  
%s
GOP:RICK SCOTT
        Candidate                Time  \
0  GOP:RICK SCOTT 2018-09-30 20:18:13   

                 

KeyboardInterrupt: 

In [ ]:
   Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Rick) == 0:
        time.sleep(60 * 15)
        Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Rick , 'GOP:RICK SCOTT' , FL)
    